# Technical Analysis

Roboquant comes out of box with support for technical analysis. It does this by having dedicate support for two popular technical analysis libraries:

1. TaLib: contains over 150 technical indicators. These are indicators ranging from RSI and Bollinger Bands to Candlestick pattern recognition. Directly using the TA-Lib library can be difficult due to its API, but roboquant makes it fairly easy by providing a convenient wrapper.
2. Ta4j: contains over 130 technical indicators and has a flexible engine to define your own trading rules.

In [1]:
%use @http://roboquant.org/roboquant.json
Welcome()

roboquant 0.9.5-SNAPSHOT 
 build: 2022-08-20T08:45:23Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 12.5.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.4 
 memory: 4096 MB 
 env: LAB

In [2]:
// We first setup the feed we'll use in all examples below
val feed = AvroFeed.sp500()

# TaLib
We start by defining the  strategy using TALib indicators we want to test in combination with the `TaLibStrategy` class. There are three steps:

1) Create an instance of `TaLibStrategy` and define how much history it should maintain before invoking the indicators. If you choose this value too low, certain indicators will fail and an exception will be thrown. For example if you want a moving average over 20 days, the history should be at least 20. If you choose the histoy value too large, it will take longer before your strategy starts to generate signals since it will wait untill it has enough history. For many of the technical indicators like candle stick patterns, 15 is sufficient (also the default value).

2) Define what you logic to use for generating BUY signals. This can be a combination of indicators and you can also include regular Kotlin code. The only restriction is that the last statement has to be a boolean; true if you want to BUY, false otherwise.

3) Define what you logic to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

Please note, that although not common practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [3]:
// Define a strategy with maximum of 15 days of history. It is important that these are enough days to calculate the various indicators.
// This example just shows how to use different indicators, they don't make much sense in this combination
val strategy = TaLibStrategy(15)

// We want to generate a BUY signal if we detect either one of the following two candlestick pattern: Morningstar or Three WhiteSoldiers
strategy.buy {
    cdlMorningStar(it) || cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows and also observe that the 5 day weighted average is below the 9 day weighted average 
strategy.sell {
    cdl2Crows(it) && ema(it.close, 5) < ema(it.close, 9)
}


Now we can use this strategy just like any other strategy. In this case we use the S&P500 feed. 

Besides strategies, we can also use TALib indicators as metrics that are captured during a run. The following code creates a custom metric that calculates the Money Flow Index (aka mfi), one of the many indicators that comes with TALib. 

In [4]:
val mfiMetric = TaLibMetric("mfi") {
    mfi(it)    
}

In [5]:
val roboquant =  Roboquant(strategy, AccountSummary(), mfiMetric)
roboquant.run(feed)

2016-01-04 - 2020-12-31 | 100% |████████████████████████████████████████| run-0 | episode=1 | phase=MAIN 


Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;) 

In [6]:
val account = roboquant.broker.account
account.summary()

account
├── last update: 2020-12-31T21:00:00Z
├── base currency: USD
├── cash: USD -723.07
├── buying power: USD -723.07
├── equity: USD 2855720.54
├── portfolio: USD 2856443.61
├── long value: USD 2856443.61
├── short value: USD 0.00
├── open positions: 89
├── unrealized p&l: USD 1610248.96
├── realized p&l: USD 245471.58
├── trades: 163
├── open orders: 0
└── closed orders: 163


In [7]:
account.portfolio.summary()

portfolio
├──          asset│  currency│      quantity│   entry price│  market price│  market value│    unrlzd p&l│
├──     STOCK:ABMD│       USD│           136│        148.24│        324.20│      44091.20│      23930.82│
├──      STOCK:ACN│       USD│             1│        189.16│        258.02│        258.02│         68.86│
├──      STOCK:AEP│       USD│           226│         88.06│         80.38│      18166.78│      -1735.00│
├──      STOCK:AES│       USD│             1│          9.54│         22.94│         22.94│         13.40│
├──      STOCK:AIG│       USD│           454│         43.79│         37.16│      16870.19│      -3010.81│
├──      STOCK:AMT│       USD│           135│        147.75│        221.21│      29863.35│       9917.42│
├──     STOCK:AMZN│       USD│             7│       2881.58│       3256.93│      22798.51│       2627.46│
├──     STOCK:ANTM│       USD│           169│        118.35│        318.13│      53763.97│      33763.12│
├──      STOCK:APA│       USD│      

Let find out which trade generated most profit (realized PnL) and plot the price and all the trades for that asset

In [8]:
// Lets find out the single most profitable trade and plot the asset and trades for that asset
val mostProfitableTrade = account.trades.maxByOrNull { it.pnl.value } !!
println(mostProfitableTrade)
val asset = mostProfitableTrade.asset
PriceBarChart(feed, asset, account.trades)

Trade(time=2020-05-07T20:00:00Z, asset=Asset(symbol=BBY, type=STOCK, currencyCode=USD, exchangeCode=NYSE, multiplier=1.0, id=), size=-846, price=73.093833, feeValue=0.0, pnlValue=40842.376686, orderId=114)


In [9]:
// It is always good to plot the equity (cash + assets) to see how the strategy performed
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)

In [10]:
// Lets also plot the Money Flow Index for one of the assets
val symbol = asset.symbol.lowercase()
val metricData = roboquant.logger.getMetric("mfi.${symbol}")
MetricChart(metricData)

## Bollinger Bands
This example is more realistic and shows how to implement a strategy based on Bollinger Bands. The underlying assumptions are:

1. If the current price is higher than the upper bound, it will continu to go up.
2. If the current price is lower than lower bound, it will continu to go down.

Of course you can easily reverse this logic from trend following to mean reversion if you prefer and compare the outcomes.

In [11]:
// Define a strategy with maximum of 20 days of history. 
val days = 20
val strategy = TaLibStrategy(days)

// We want to BUY if current price is higher than the Bollinger Bands higher bound 
strategy.buy {
    val (h, m, l) = bbands(it, days)
    it.close.last() > h 
}

// We want to SELL if current price is lower than the Bollinger Bands lower bound 
strategy.sell {
    val (h, m, l) = bbands(it, days)
    it.close.last() < l   
}

In [12]:
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed)

2016-01-04 - 2020-12-31 | 100% |████████████████████████████████████████| run-0 | episode=1 | phase=MAIN 


In [13]:
val equity = roboquant.logger.getMetric("account.equity")
MetricChart(equity)

# Ta4j

We'll now will define a strategy using indicators and rules found in the `ta4j` library and the roboquant `Ta4jStrategy` strategy class. The `Ta4jStrategy` allows to define flexible rules to generate BUY or SELL signals for assets using standard `ta4j` indicators and rules. Please note that indicators and rules from `ta4j` can be freely used and mixed, but defining strategies and back tests are done using roboquant constructs like the `Ta4jStrategy`, so not the `ta4j` back-test framework.

You can find out more on using `ta4j` on the wiki site  https://ta4j.github.io/ta4j-wiki/Trading-strategies.html or visit the `ta4j` Discord server. 

The main steps to follow are:

1. Import the ta4j packages that you want to use (in future releases of roboquant this might be done automatically) 
2. Create a `Ta4jStrategy` instance and (optional) indicate how much history we want to track
3. Define the buying and selling rules
4. Run a back test and evaluate the results (this is not specific to `ta4j` based stategies)

<div class="alert alert-block alert-info">
If you are coming from `ta4j` please note: an important difference with `ta4j` back testing and `roboquant` is that in roboqaunt by default the buying and selling rules apply to all assets found in a feed. So in the below example the defined strategy is applied to all the S&P 500 stocks and not just a single stock. 
</div>

In [14]:
import org.ta4j.core.indicators.*
import org.ta4j.core.rules.*
import org.ta4j.core.indicators.helpers.*

In [15]:
val strategy = Ta4jStrategy(maxBarCount = 30)

strategy.buy { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedUpIndicatorRule(shortSma, longSma)
}

strategy.sell { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedDownIndicatorRule(shortSma, longSma)
        .or(StopLossRule(closePrice, 3.0))
        .or(StopGainRule(closePrice, 2.0))
}

In [16]:
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed)

2016-01-04 - 2020-12-31 | 100% |████████████████████████████████████████| run-0 | episode=1 | phase=MAIN 


In [17]:
roboquant.broker.account.summary()

account
├── last update: 2020-12-31T21:00:00Z
├── base currency: USD
├── cash: USD 24379.98
├── buying power: USD 24379.98
├── equity: USD 2202803.62
├── portfolio: USD 2178423.64
├── long value: USD 2178423.64
├── short value: USD 0.00
├── open positions: 121
├── unrealized p&l: USD 352532.77
├── realized p&l: USD 850270.85
├── trades: 7115
├── open orders: 0
└── closed orders: 7115


In [18]:
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)

## Bollinger Bands

This is a similar Bollinger Bands strategy as the one above for Ta-Lib. 

In [19]:
import org.ta4j.core.indicators.bollinger.*
import org.ta4j.core.indicators.statistics.*

// How big a lookback period should we use
val period = 20

val strategy = Ta4jStrategy(maxBarCount = period)

strategy.buy { series ->
    val closePrice = ClosePriceIndicator(series)
    val sma = SMAIndicator(closePrice, period)
    val bm = BollingerBandsMiddleIndicator(sma)
    val sd = StandardDeviationIndicator(closePrice, period);
    val bu = BollingerBandsUpperIndicator(bm, sd)
    CrossedUpIndicatorRule(closePrice, bu)
}

strategy.sell { series ->
    val closePrice = ClosePriceIndicator(series)
    val sma = SMAIndicator(closePrice, period)
    val bm = BollingerBandsMiddleIndicator(sma)
    val sd = StandardDeviationIndicator(closePrice, period);
    val bl = BollingerBandsLowerIndicator(bm, sd)
    CrossedDownIndicatorRule(closePrice, bl)
}


In [20]:
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed)

2016-01-04 - 2020-12-31 | 100% |████████████████████████████████████████| run-0 | episode=1 | phase=MAIN 


In [21]:
roboquant.broker.account.summary()

account
├── last update: 2020-12-31T21:00:00Z
├── base currency: USD
├── cash: USD -164.62
├── buying power: USD -164.62
├── equity: USD 2151673.75
├── portfolio: USD 2151838.36
├── long value: USD 2151838.36
├── short value: USD 0.00
├── open positions: 129
├── unrealized p&l: USD 357338.75
├── realized p&l: USD 794335.00
├── trades: 4539
├── open orders: 0
└── closed orders: 4539


In [22]:
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)